# Controle de Qualidade do Array
Autores: Rafaella Ormond e José Jaime Martinez-Magaña <br>
***Descrição:***<br>
Este script prepara os arquivos para a imputação.

**ENTRADA:** Arquivos PLINK (.bed/.bim/.fam)<br>
**SAÍDA:**  
1) Arquivos de resultados de PCA (tabela de componentes principais para as amostras).  
2) Arquivos da Matriz de Relações Genéticas (GRM).

### ***Requisitos:***
### Download do Plink
É possível baixar o Plink nas versões 1.9 e 2.0 seguindo as instruções do site oficial.<br>
Para instalar o plink2 [acesse aqui](https://www.cog-genomics.org/plink/2.0/)<br>
Para instalar o plink1.9 [acesse aqui](https://www.cog-genomics.org/plink/1.9/)<br>

### Download do BCFtools:
Para instalar o bcftools [acesse aqui](http://samtools.github.io/bcftools/)<br>

### Genoma de Referência: 
Nas primeiras etapas, o genoma de referência depende do array utilizado, podendo ser **hg19** ou **hg38**.<br>
Para baixar o genoma de referência **hg19**, use o arquivo: [hg19.fa.gz](https://hgdownload.soe.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz)<br> 
Para baixar o genoma de referência **hg38**, use o arquivo: [hg38.fa.gz](https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz)


### 1) Configuração do Usuário
***Descrição:***<br>
Defina os parâmetros e ajuste os caminhos e arquivos de acordo com a sua análise.

In [ ]:
# CONFIGURAÇÃO DO USUÁRIO - MODIFIQUE DE ACORDO COM SUA ANÁLISE

# Prefixo dos arquivos PLINK (.bed, .bim, .fam)
inPath="/caminho_para_seus_dados/prefixo_plink"

# Nome da sua coorte
cohortName="nome_da_coorte"

# Diretório para os arquivos VCF
clvcfPath="caminho_arquivos_vcf"

# Diretório para os VCFs por cromossomo
clvcfbychrPath="caminho_vcf_por_cromossomo"

# Arquivo do genoma de referência (hg19 ou hg38)
refFile="/caminho/para/referencia/hg19.fa"

### 2) Converter arquivos PLINK para VCF, realizar controle de qualidade e indexar os arquivos
***Descrição:***<br>
Converte arquivos binários do PLINK para o formato VCF compactado e cria arquivos de índice para acesso eficiente.
Filtros de controle de qualidade são aplicados durante a conversão para excluir variantes e amostras com base na frequência do alelo minoritário (MAF), equilíbrio de Hardy-Weinberg (HWE), e proporção de genótipos ausentes por variante e por indivíduo.

Ajuste os caminhos e nomes dos arquivos conforme sua análise.

In [ ]:
plink --bfile ${inPath} --recode vcf-iid bgz --maf 0.01 --hwe 1e-6 --geno 0.05 --mind 0.05 --out ${clvcfPath}/${cohortName}
bcftools index ${clvcfPath}/${cohortName}.vcf.gz

### 3) Alinhamento
***Descrição:***<br>
Alinha os alelos do arquivo VCF com o genoma de referência. Ajuste o número de threads conforme a capacidade do seu sistema.

In [ ]:
bcftools convert ${clvcfPath}/${cohortName}.vcf.gz -Ob -o ${clvcfPath}/${cohortName}.bcf.gz --threads 25
bcftools +fixref ${clvcfPath}/${cohortName}.bcf.gz -- -f ${refFile}
bcftools +fixref ${clvcfPath}/${cohortName}.bcf.gz -Oz -o ${clvcfPath}/${cohortName}.vcf.gz -- -d -m flip -f ${refFile}

### 4) Dividir o arquivo VCF por cromossomo
***Descrição:***<br>
Divide o arquivo VCF por cromossomo para imputação e ordena os arquivos VCF por cromossomo, garantindo a ordem correta antes das análises subsequentes.

In [ ]:
for chr_num in {1..22}
do
  plink2 --vcf ${clvcfPath}/${cohortName}.vcf.gz --chr ${chr_num} --recode vcf-4.2 bgz --out ${clvcfbychrPath}/${cohortName}_forimputation_chr${chr_num}
done

for chr_num in {1..22}
do
  bcftools sort ${clvcfbychrPath}/${cohortName}_forimputation_chr${chr_num}.vcf.gz -Oz -o ${clvcfbychrPath}/${cohortName}_forimputation_chr${chr_num}_sorted.vcf.gz
done

### 5) Instruções para Imputação no TOPMed

Por favor, acesse o [TOPMed Imputation Server](https://imputation.biodatacatalyst.nhlbi.nih.gov).

Faça login na sua conta e acesse o painel de controle.  
No topo da página, clique em **“Run” → “Genotype Imputation (Minimac4)”**.
Faça o upload dos seus arquivos VCF e configure as seguintes opções:

- **Array Build:** selecione *hg19* ou *hg38* (de acordo com a build do seu arquivo).  
- **Rsq Filter:** *desligado*  
- **Phasing:** *Eagle v2.4 (saída faseada)*  
- **Population:** selecione *Todas as populações* (atualmente “vs. TOPMed Panel”)  
- **Mode:** *Quality Control & Imputation*  
- Marque a opção **“Generate Meta-imputation file”**
Após enviar o trabalho, você receberá um e-mail quando os resultados estiverem prontos.  
Você também receberá uma **senha por e-mail** para acessar as pastas de saída.  
Faça o download de todas as pastas de resultados para o seu diretório de trabalho.
